In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import json


df = pd.read_csv("../data/raw/retail_profit_margin_dataset_30k.csv")
df.head()

,product_id,product_name,category,brand,store_id,store_name,store_location,base_price,discount_rate,promotion_type,day_of_year,month,day_of_week,season,is_holiday,avg_units_sold_30d,avg_customers_30d,profit_margin,profit_class
0,P100901,L'Oreal Sports Item,Sports,L'Oreal,S010,Amazon Store 10,US-East,211.96,0.00,No Promotion,168,6,Tue,Spring,0,14,33,0.332,high
1,P100424,Samsung Electronics Item,Electronics,Samsung,S035,Amazon Store 35,US-West,231.01,0.35,Flash Sale,339,12,Wed,Fall,0,25,35,-0.136,loss
2,P100014,Adidas Home Item,Home,Adidas,S029,Amazon Store 29,US-East,147.61,0.00,No Promotion,193,7,Sat,Summer,0,8,11,0.238,medium
3,P100848,HP Grocery Item,Grocery,HP,S049,Amazon Store 49,US-East,23.40,0.58,Clearance,283,10,Mon,Fall,0,23,39,-0.500,loss
4,P100122,AmazonBasics Grocery Item,Grocery,AmazonBasics,S002,Amazon Store 2,US-East,29.66,0.11,Holiday,336,12,Mon,Fall,0,32,45,0.247,medium


In [3]:
# Fix missing promotion_type
df["promotion_type"] = df["promotion_type"].fillna("No Promotion")




In [4]:
q1 = df["profit_margin"].quantile(0.33)
q2 = df["profit_margin"].quantile(0.66)

def profit_bucket_quantile(x):
    if x < 0:
        return "loss"
    elif x < q1:
        return "low"
    elif x < q2:
        return "medium"
    else:
        return "high"
    
df["profit_class_quantile"] = df["profit_margin"].apply(profit_bucket_quantile)

df["profit_class_quantile"].value_counts()

profit_class_quantile
high      10245
medium     9858
loss       5202
low        4695
Name: count, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

# Features and target
X = df.drop(columns=["profit_margin","profit_class", "profit_class_quantile", "product_name",
    "store_name"])
y = df["profit_class_quantile"]  # use string labels for now

# First split: train + temp (val+test)
X_train, X_temp, y_train, y_temp = train_test_split(
    X,
    y,
    test_size=0.3,
    stratify=y,
    random_state=42
)

# Second split: validation + test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    stratify=y_temp,
    random_state=42
)

# Print sizes
print("Train:", X_train.shape, "Validation:", X_val.shape, "Test:", X_test.shape)


Train: (21000, 15) Validation: (4500, 15) Test: (4500, 15)


In [6]:
df["profit_margin"].describe()


count    30000.000000
mean         0.190109
std          0.228891
min         -0.500000
25%          0.090000
50%          0.245000
75%          0.351000
max          0.500000
Name: profit_margin, dtype: float64

In [7]:
from sklearn.utils import resample
import pandas as pd

# Combine X and y for resampling
train_df = X_train.copy()
train_df["profit_class_quantile"] = y_train.values

# Separate classes
df_loss   = train_df[train_df["profit_class_quantile"] == "loss"]
df_low    = train_df[train_df["profit_class_quantile"] == "low"]
df_medium = train_df[train_df["profit_class_quantile"] == "medium"]
df_high   = train_df[train_df["profit_class_quantile"] == "high"]

# Target size = size of medium class
target_size = len(df_medium)

# Upsample minority classes
df_loss_up = resample(
    df_loss,
    replace=True,
    n_samples=target_size,
    random_state=42
)

df_low_up = resample(
    df_low,
    replace=True,
    n_samples=target_size,
    random_state=42
)

# Combine back
train_balanced = pd.concat(
    [df_loss_up, df_low_up, df_medium, df_high],
    axis=0
).sample(frac=1, random_state=42)  # shuffle

# Split back to X / y
X_train_bal = train_balanced.drop(columns=["profit_class_quantile"])
y_train_bal = train_balanced["profit_class_quantile"]

# Check distribution
print(y_train_bal.value_counts())


profit_class_quantile
high      7172
loss      6901
low       6901
medium    6901
Name: count, dtype: int64


In [8]:
import pandas as pd
import numpy as np

# Simple feature engineering
def create_features(df):
    df = df.copy()
    
    # Price features
    df['actual_price'] = df['base_price'] * (1 - df['discount_rate'])
    df['discount_amount'] = df['base_price'] * df['discount_rate']
    
    # Sales features
    df['units_per_customer'] = df['avg_units_sold_30d'] / (df['avg_customers_30d'] + 1)
    
    return df

# Apply to train and test
X_train_bal_fe = create_features(X_train_bal)
X_test_fe = create_features(X_test)

# Define categorical columns for CatBoost
categorical_features = ['product_id', 'product_name', 'category', 'brand',
                       'store_id', 'store_name', 'store_location',
                       'promotion_type', 'day_of_week', 'season']

print("Done. Ready for model training.")

Done. Ready for model training.


In [9]:
print(X_train_bal_fe.columns.tolist())
print(X_train_bal_fe.dtypes)

['product_id', 'category', 'brand', 'store_id', 'store_location', 'base_price', 'discount_rate', 'promotion_type', 'day_of_year', 'month', 'day_of_week', 'season', 'is_holiday', 'avg_units_sold_30d', 'avg_customers_30d', 'actual_price', 'discount_amount', 'units_per_customer']
product_id             object
category               object
brand                  object
store_id               object
store_location         object
base_price            float64
discount_rate         float64
promotion_type         object
day_of_year             int64
month                   int64
day_of_week            object
season                 object
is_holiday              int64
avg_units_sold_30d      int64
avg_customers_30d       int64
actual_price          float64
discount_amount       float64
units_per_customer    float64
dtype: object


In [10]:
# Remove profit_class if it exists
if 'profit_class' in X_train_bal_fe.columns:
    X_train_bal_fe = X_train_bal_fe.drop(columns=['profit_class'])
    
if 'profit_class' in X_test_fe.columns:
    X_test_fe = X_test_fe.drop(columns=['profit_class'])

print("Done. Cleaned both datasets.")

Done. Cleaned both datasets.


In [21]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, log_loss, classification_report, confusion_matrix
import numpy as np

# Get categorical column indices
cat_features = ['product_id', 'category', 'brand',
                'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']

# Find the indices of categorical columns
cat_feature_indices = [X_train_bal_fe.columns.get_loc(col) for col in cat_features if col in X_train_bal_fe.columns]

# Initialize CatBoost
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    loss_function='MultiClass',
    eval_metric='TotalF1',
    random_seed=42,
    verbose=100,
    cat_features=cat_feature_indices
)

# Cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_f1_scores = []
cv_acc_scores = []

print("Starting 5-Fold Cross-Validation...\n")

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train_bal_fe, y_train_bal), 1):
    print(f"Fold {fold}/5")
    
    X_tr, X_val = X_train_bal_fe.iloc[train_idx], X_train_bal_fe.iloc[val_idx]
    y_tr, y_val = y_train_bal.iloc[train_idx], y_train_bal.iloc[val_idx]
    
    model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average='macro')
    acc = accuracy_score(y_val, y_pred)
    
    cv_f1_scores.append(f1)
    cv_acc_scores.append(acc)
    
    print(f"Fold {fold} - F1: {f1:.4f}, Accuracy: {acc:.4f}\n")

print(f"Mean CV F1 Score: {np.mean(cv_f1_scores):.4f} (+/- {np.std(cv_f1_scores):.4f})")
print(f"Mean CV Accuracy: {np.mean(cv_acc_scores):.4f} (+/- {np.std(cv_acc_scores):.4f})")

# Train final model on full training data
print("\nTraining final model on full training set...")
model.fit(X_train_bal_fe, y_train_bal, verbose=100)

# Evaluate on test set
y_pred_test = model.predict(X_test_fe)
y_pred_proba = model.predict_proba(X_test_fe)

test_f1 = f1_score(y_test, y_pred_test, average='macro')
test_acc = accuracy_score(y_test, y_pred_test)
test_loss = log_loss(y_test, y_pred_proba)

print("\n" + "="*50)
print("TEST SET RESULTS:")
print("="*50)
print(f"Accuracy: {test_acc:.4f}")
print(f"F1 Score (Macro): {test_f1:.4f}")
print(f"Log Loss: {test_loss:.4f}")

print("\n" + "="*50)
print("CLASSIFICATION REPORT:")
print("="*50)
print(classification_report(y_test, y_pred_test))

print("="*50)
print("CONFUSION MATRIX:")
print("="*50)
print(confusion_matrix(y_test, y_pred_test))

Starting 5-Fold Cross-Validation...

Fold 1/5
Fold 1 - F1: 0.7483, Accuracy: 0.7455

Fold 2/5
Fold 2 - F1: 0.7105, Accuracy: 0.7062

Fold 3/5
Fold 3 - F1: 0.7378, Accuracy: 0.7354

Fold 4/5
Fold 4 - F1: 0.7432, Accuracy: 0.7404

Fold 5/5
Fold 5 - F1: 0.7377, Accuracy: 0.7396

Mean CV F1 Score: 0.7355 (+/- 0.0131)
Mean CV Accuracy: 0.7334 (+/- 0.0140)

Training final model on full training set...
0:	learn: 0.5698325	total: 97.6ms	remaining: 48.7s
100:	learn: 0.7427289	total: 8.93s	remaining: 35.3s
200:	learn: 0.7526858	total: 18.6s	remaining: 27.6s
300:	learn: 0.7628331	total: 27.6s	remaining: 18.2s
400:	learn: 0.7754366	total: 36.6s	remaining: 9.03s
499:	learn: 0.7874684	total: 45.4s	remaining: 0us

TEST SET RESULTS:
Accuracy: 0.5453
F1 Score (Macro): 0.4760
Log Loss: 0.8998

CLASSIFICATION REPORT:
              precision    recall  f1-score   support

        high       0.61      0.76      0.68      1537
        loss       0.78      0.71      0.75       780
         low       0.33    

In [22]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

# Get categorical column indices
cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']
cat_feature_indices = [X_train_bal_fe.columns.get_loc(col) for col in cat_features if col in X_train_bal_fe.columns]

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 300, 800),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'random_strength': trial.suggest_float('random_strength', 0, 10),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0, 1),
        'loss_function': 'MultiClass',
        'eval_metric': 'TotalF1',
        'random_seed': 42,
        'verbose': False,
        'cat_features': cat_feature_indices
    }
    
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    cv_scores = []
    
    for train_idx, val_idx in cv.split(X_train_bal_fe, y_train_bal):
        X_tr, X_val = X_train_bal_fe.iloc[train_idx], X_train_bal_fe.iloc[val_idx]
        y_tr, y_val = y_train_bal.iloc[train_idx], y_train_bal.iloc[val_idx]
        
        model = CatBoostClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
        
        y_pred = model.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='macro')
        cv_scores.append(f1)
    
    return np.mean(cv_scores)

print("Starting hyperparameter optimization (50 trials)...")
print("This may take 15-30 minutes...\n")

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True)

print("\n" + "="*50)
print("BEST HYPERPARAMETERS:")
print("="*50)
print(study.best_params)
print(f"\nBest CV F1 Score: {study.best_value:.4f}")

# Train final model with best parameters
print("\n" + "="*50)
print("Training final model with best parameters...")
print("="*50)

best_params = study.best_params
best_params['loss_function'] = 'MultiClass'
best_params['eval_metric'] = 'TotalF1'
best_params['random_seed'] = 42
best_params['verbose'] = 100
best_params['cat_features'] = cat_feature_indices

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train_bal_fe, y_train_bal)

# Evaluate
from sklearn.metrics import accuracy_score, log_loss, classification_report, confusion_matrix

y_pred_test = final_model.predict(X_test_fe)
y_pred_proba = final_model.predict_proba(X_test_fe)

test_f1 = f1_score(y_test, y_pred_test, average='macro')
test_acc = accuracy_score(y_test, y_pred_test)
test_loss = log_loss(y_test, y_pred_proba)

print("\n" + "="*50)
print("TUNED MODEL - TEST SET RESULTS:")
print("="*50)
print(f"Accuracy: {test_acc:.4f}")
print(f"F1 Score (Macro): {test_f1:.4f}")
print(f"Log Loss: {test_loss:.4f}")

print("\n" + "="*50)
print("CLASSIFICATION REPORT:")
print("="*50)
print(classification_report(y_test, y_pred_test))

print("="*50)
print("CONFUSION MATRIX:")
print("="*50)
print(confusion_matrix(y_test, y_pred_test))

[I 2025-12-27 19:59:00,378] A new study created in memory with name: no-name-3aa3e3b8-455e-4d55-a320-604c54a6b13a


Starting hyperparameter optimization (50 trials)...
This may take 15-30 minutes...



  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-12-27 19:59:36,067] Trial 0 finished with value: 0.7258198151753793 and parameters: {'iterations': 726, 'learning_rate': 0.09228716181658375, 'depth': 4, 'l2_leaf_reg': 1.9634180895858169, 'min_data_in_leaf': 25, 'random_strength': 3.185693805681514, 'bagging_temperature': 0.9043203123343585}. Best is trial 0 with value: 0.7258198151753793.
[I 2025-12-27 19:59:52,539] Trial 1 finished with value: 0.7227750501984507 and parameters: {'iterations': 625, 'learning_rate': 0.2619653979786518, 'depth': 4, 'l2_leaf_reg': 2.6402266781380566, 'min_data_in_leaf': 47, 'random_strength': 3.335058172422656, 'bagging_temperature': 0.31653060663520516}. Best is trial 0 with value: 0.7258198151753793.
[I 2025-12-27 20:00:22,787] Trial 2 finished with value: 0.7285454435894693 and parameters: {'iterations': 327, 'learning_rate': 0.2493906296934598, 'depth': 5, 'l2_leaf_reg': 5.918379329432392, 'min_data_in_leaf': 6, 'random_strength': 7.3530786657702425, 'bagging_temperature': 0.4916301029942548

KeyboardInterrupt: 

In [11]:
print("Test set distribution:")
print(y_test.value_counts())
print("\nTrain set distribution:")
print(y_train_bal.value_counts())

Test set distribution:
profit_class_quantile
high      1537
medium    1478
loss       780
low        705
Name: count, dtype: int64

Train set distribution:
profit_class_quantile
high      7172
loss      6901
low       6901
medium    6901
Name: count, dtype: int64


In [12]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights from original imbalanced training data
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Create dictionary
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Class weights:", class_weight_dict)

# Train CatBoost with class weights (NO resampling)
from catboost import CatBoostClassifier

cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features if col in X_train.columns]

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    class_weights=class_weight_dict,
    cat_features=cat_feature_indices,
    random_seed=42,
    verbose=100
)

# Use ORIGINAL unbalanced training data
model.fit(X_train, y_train)

# Predict on test
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report, accuracy_score, f1_score
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1:", f1_score(y_test, y_pred, average='macro'))
print("\n", classification_report(y_test, y_pred))

Class weights: {'high': np.float64(0.7320133853876185), 'loss': np.float64(1.441911562757484), 'low': np.float64(1.5976871576384661), 'medium': np.float64(0.7607593102448921)}
0:	learn: 1.2945069	total: 205ms	remaining: 1m 42s
100:	learn: 0.7788175	total: 8.41s	remaining: 33.2s
200:	learn: 0.7446615	total: 17.8s	remaining: 26.4s
300:	learn: 0.7147076	total: 26s	remaining: 17.2s
400:	learn: 0.6869111	total: 34.9s	remaining: 8.61s
499:	learn: 0.6606290	total: 42.1s	remaining: 0us

Test Accuracy: 0.5395555555555556
Test F1: 0.5252926000212195

               precision    recall  f1-score   support

        high       0.62      0.74      0.67      1537
        loss       0.70      0.81      0.75       780
         low       0.35      0.62      0.44       705
      medium       0.45      0.16      0.23      1478

    accuracy                           0.54      4500
   macro avg       0.53      0.58      0.53      4500
weighted avg       0.53      0.54      0.51      4500



In [23]:
print("Training set distribution:")
print(y_train.value_counts())
print("\nTraining percentages:")
print(y_train.value_counts(normalize=True) * 100)

print("\n" + "="*50)
print("Test set distribution:")
print(y_test.value_counts())
print("\nTest percentages:")
print(y_test.value_counts(normalize=True) * 100)

Training set distribution:
profit_class_3
profit_high    10287
profit_low      7072
loss            3641
Name: count, dtype: int64

Training percentages:
profit_class_3
profit_high    48.985714
profit_low     33.676190
loss           17.338095
Name: proportion, dtype: float64

Test set distribution:
profit_class_3
profit_high    2204
profit_low     1515
loss            781
Name: count, dtype: int64

Test percentages:
profit_class_3
profit_high    48.977778
profit_low     33.666667
loss           17.355556
Name: proportion, dtype: float64


In [26]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]

model = CatBoostClassifier(
    iterations=300,  # Reduced from 500
    learning_rate=0.05,  # Slower learning
    depth=4,  # Shallower trees
    l2_leaf_reg=10,  # Strong regularization
    min_data_in_leaf=50,  # More data per leaf
    class_weights=class_weight_dict,
    cat_features=cat_feature_indices,
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1:", f1_score(y_test, y_pred, average='macro'))
print("\n", classification_report(y_test, y_pred))

0:	learn: 1.3421211	total: 66.8ms	remaining: 20s
100:	learn: 0.8039161	total: 4.7s	remaining: 9.27s
200:	learn: 0.7923870	total: 9.76s	remaining: 4.81s
299:	learn: 0.7827788	total: 14.7s	remaining: 0us

Test Accuracy: 0.5457777777777778
Test F1: 0.5362971024877657

               precision    recall  f1-score   support

        high       0.63      0.71      0.67      1537
        loss       0.71      0.84      0.77       780
         low       0.36      0.63      0.45       705
      medium       0.45      0.18      0.25      1478

    accuracy                           0.55      4500
   macro avg       0.54      0.59      0.54      4500
weighted avg       0.54      0.55      0.52      4500



In [27]:
# Enhanced feature engineering
def create_better_features(df):
    df = df.copy()
    
    # Price features
    df['actual_price'] = df['base_price'] * (1 - df['discount_rate'])
    df['discount_amount'] = df['base_price'] * df['discount_rate']
    df['price_per_unit_sold'] = df['base_price'] / (df['avg_units_sold_30d'] + 1)
    
    # Revenue features
    df['total_revenue'] = df['actual_price'] * df['avg_units_sold_30d']
    df['revenue_per_customer'] = df['total_revenue'] / (df['avg_customers_30d'] + 1)
    
    # Efficiency features
    df['units_per_customer'] = df['avg_units_sold_30d'] / (df['avg_customers_30d'] + 1)
    df['customer_to_units_ratio'] = df['avg_customers_30d'] / (df['avg_units_sold_30d'] + 1)
    
    # Discount features
    df['has_discount'] = (df['discount_rate'] > 0).astype(int)
    df['high_discount'] = (df['discount_rate'] > 0.3).astype(int)
    
    return df

# Apply to train and test
X_train_fe = create_better_features(X_train)
X_test_fe = create_better_features(X_test)

# Train model
cat_feature_indices = [X_train_fe.columns.get_loc(col) for col in cat_features]

model = CatBoostClassifier(
    iterations=300,
    learning_rate=0.05,
    depth=5,
    l2_leaf_reg=8,
    min_data_in_leaf=40,
    class_weights=class_weight_dict,
    cat_features=cat_feature_indices,
    random_seed=42,
    verbose=100
)

model.fit(X_train_fe, y_train)
y_pred = model.predict(X_test_fe)

from sklearn.metrics import classification_report, accuracy_score, f1_score
print("\nTest Accuracy:", accuracy_score(y_test, y_pred))
print("Test F1:", f1_score(y_test, y_pred, average='macro'))
print("\n", classification_report(y_test, y_pred))

0:	learn: 1.3429992	total: 62.3ms	remaining: 18.6s
100:	learn: 0.8024017	total: 6.84s	remaining: 13.5s
200:	learn: 0.7880780	total: 13.7s	remaining: 6.76s
299:	learn: 0.7761722	total: 20.5s	remaining: 0us

Test Accuracy: 0.5446666666666666
Test F1: 0.5319873984522631

               precision    recall  f1-score   support

        high       0.63      0.72      0.67      1537
        loss       0.69      0.85      0.76       780
         low       0.35      0.59      0.44       705
      medium       0.45      0.18      0.25      1478

    accuracy                           0.54      4500
   macro avg       0.53      0.59      0.53      4500
weighted avg       0.54      0.54      0.51      4500



In [19]:
# Check profit margin ranges for each class
print("Profit margin ranges by class:")
print(df.groupby('profit_class_quantile')['profit_margin'].agg(['min', 'max', 'mean', 'std']))

print("\n" + "="*50)
print("Class boundaries:")
print(f"loss: < 0")
print(f"low: 0 to {df[df['profit_margin'] >= 0]['profit_margin'].quantile(0.33):.4f}")
print(f"medium: {df[df['profit_margin'] >= 0]['profit_margin'].quantile(0.33):.4f} to {df[df['profit_margin'] >= 0]['profit_margin'].quantile(0.66):.4f}")
print(f"high: > {df[df['profit_margin'] >= 0]['profit_margin'].quantile(0.66):.4f}")

Profit margin ranges by class:
                         min    max      mean       std
profit_class_quantile                                  
high                   0.312  0.500  0.394898  0.053539
loss                  -0.500 -0.001 -0.218517  0.164788
low                    0.000  0.158  0.086637  0.045453
medium                 0.159  0.311  0.242190  0.040426

Class boundaries:
loss: < 0
low: 0 to 0.2240
medium: 0.2240 to 0.3370
high: > 0.3370


In [18]:
# Check how you actually created the classes
print("How many samples in original df:", len(df))
print("\nActual value counts:")
print(df['profit_class_quantile'].value_counts().sort_index())

# Verify the thresholds you used
non_negative = df[df['profit_margin'] >= 0]['profit_margin']
q1_actual = non_negative.quantile(0.33)
q2_actual = non_negative.quantile(0.66)

print(f"\nQuantiles calculated from non-negative values:")
print(f"Q1 (33rd percentile): {q1_actual:.4f}")
print(f"Q2 (66th percentile): {q2_actual:.4f}")

# Check a few examples around boundaries
print("\nSamples near low/medium boundary:")
print(df[(df['profit_margin'] > 0.15) & (df['profit_margin'] < 0.17)][['profit_margin', 'profit_class_quantile']].head(10))

How many samples in original df: 30000

Actual value counts:
profit_class_quantile
high      10245
loss       5202
low        4695
medium     9858
Name: count, dtype: int64

Quantiles calculated from non-negative values:
Q1 (33rd percentile): 0.2240
Q2 (66th percentile): 0.3370

Samples near low/medium boundary:
     profit_margin profit_class_quantile
14           0.165                medium
50           0.166                medium
97           0.152                   low
175          0.161                medium
179          0.165                medium
347          0.154                   low
361          0.151                   low
404          0.153                   low
447          0.158                   low
563          0.167                medium


In [17]:
# Recalculate quantiles correctly
non_negative = df[df['profit_margin'] >= 0]['profit_margin']
q1 = non_negative.quantile(0.33)
q2 = non_negative.quantile(0.66)

print(f"Correct thresholds: q1={q1:.4f}, q2={q2:.4f}")

# Recreate classes
def profit_bucket_correct(x):
    if x < 0:
        return "loss"
    elif x < q1:
        return "low"
    elif x < q2:
        return "medium"
    else:
        return "high"

# Apply correct bucketing
df['profit_class_correct'] = df['profit_margin'].apply(profit_bucket_correct)

print("\nNew class distribution:")
print(df['profit_class_correct'].value_counts().sort_index())

# Now redo everything with correct classes
X = df.drop(columns=["profit_margin", "profit_class", "profit_class_quantile", 
                     "profit_class_correct", "product_name", "store_name"])
y = df["profit_class_correct"]

# Split again
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("\nTrain:", X_train.shape, "Test:", X_test.shape)

Correct thresholds: q1=0.2240, q2=0.3370

New class distribution:
profit_class_correct
high      8491
loss      5202
low       8108
medium    8199
Name: count, dtype: int64

Train: (21000, 16) Test: (4500, 16)


In [31]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# Define categorical features
cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]

# Train model
model = CatBoostClassifier(
    iterations=400,
    learning_rate=0.08,
    depth=6,
    l2_leaf_reg=5,
    class_weights=class_weight_dict,
    cat_features=cat_feature_indices,
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)

print("\n" + "="*50)
print("TEST RESULTS WITH CORRECT CLASSES:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='macro'):.4f}")
print("\n", classification_report(y_test, y_pred))

0:	learn: 1.3173124	total: 71.9ms	remaining: 28.7s
100:	learn: 0.8343247	total: 7.85s	remaining: 23.3s
200:	learn: 0.8101235	total: 16.1s	remaining: 16s
300:	learn: 0.7917254	total: 24.3s	remaining: 7.99s
399:	learn: 0.7738830	total: 32.3s	remaining: 0us

TEST RESULTS WITH CORRECT CLASSES:
Accuracy: 0.5547
F1 Score: 0.5150

               precision    recall  f1-score   support

        high       0.52      0.82      0.64      1274
        loss       0.71      0.87      0.78       780
         low       0.51      0.58      0.54      1216
      medium       0.41      0.05      0.10      1230

    accuracy                           0.55      4500
   macro avg       0.54      0.58      0.52      4500
weighted avg       0.52      0.55      0.49      4500



In [32]:
# Create 3 classes
def profit_bucket_3class(x):
    if x < 0:
        return "loss"
    elif x < 0.25:  # Combine low and medium
        return "profit_low"
    else:
        return "profit_high"

# Apply new bucketing
df['profit_class_3'] = df['profit_margin'].apply(profit_bucket_3class)

print("New 3-class distribution:")
print(df['profit_class_3'].value_counts().sort_index())

# Prepare data
X = df.drop(columns=["profit_margin", "profit_class", "profit_class_quantile", 
                     "profit_class_correct", "profit_class_3", "product_name", "store_name"])
y = df["profit_class_3"]

# Split
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("\nTrain:", X_train.shape, "Test:", X_test.shape)
print("\nTrain distribution:")
print(y_train.value_counts())
print("\nTest distribution:")
print(y_test.value_counts())

New 3-class distribution:
profit_class_3
loss            5202
profit_high    14695
profit_low     10103
Name: count, dtype: int64

Train: (21000, 15) Test: (4500, 15)

Train distribution:
profit_class_3
profit_high    10287
profit_low      7072
loss            3641
Name: count, dtype: int64

Test distribution:
profit_class_3
profit_high    2204
profit_low     1515
loss            781
Name: count, dtype: int64


In [33]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))
print("Class weights:", class_weight_dict)

# Define categorical features
cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                'promotion_type', 'day_of_week', 'season']
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]

# Train model
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    l2_leaf_reg=3,
    class_weights=class_weight_dict,
    cat_features=cat_feature_indices,
    random_seed=42,
    verbose=100
)

model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)

print("\n" + "="*50)
print("3-CLASS MODEL RESULTS:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"F1 Score (Macro): {f1_score(y_test, y_pred, average='macro'):.4f}")
print("\n", classification_report(y_test, y_pred))

Class weights: {'loss': np.float64(1.9225487503433123), 'profit_high': np.float64(0.6804704967434626), 'profit_low': np.float64(0.9898190045248869)}
0:	learn: 1.0212572	total: 46.3ms	remaining: 23.1s
100:	learn: 0.5596422	total: 6.39s	remaining: 25.2s
200:	learn: 0.5362747	total: 13.1s	remaining: 19.4s
300:	learn: 0.5158160	total: 19.6s	remaining: 13s
400:	learn: 0.4979527	total: 26.4s	remaining: 6.51s
499:	learn: 0.4796469	total: 33.1s	remaining: 0us

3-CLASS MODEL RESULTS:
Accuracy: 0.7156
F1 Score (Macro): 0.7095

               precision    recall  f1-score   support

        loss       0.70      0.86      0.77       781
 profit_high       0.80      0.79      0.80      2204
  profit_low       0.59      0.54      0.56      1515

    accuracy                           0.72      4500
   macro avg       0.70      0.73      0.71      4500
weighted avg       0.71      0.72      0.71      4500



In [34]:
from sklearn.model_selection import StratifiedKFold

# Cross-validation first
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = []

print("Running 5-Fold Cross-Validation...")
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model_cv = CatBoostClassifier(
        iterations=500,
        learning_rate=0.1,
        depth=6,
        l2_leaf_reg=3,
        class_weights=class_weight_dict,
        cat_features=cat_feature_indices,
        random_seed=42,
        verbose=False
    )
    
    model_cv.fit(X_tr, y_tr, eval_set=(X_val, y_val), early_stopping_rounds=50, verbose=False)
    y_pred_val = model_cv.predict(X_val)
    f1 = f1_score(y_val, y_pred_val, average='macro')
    cv_scores.append(f1)
    print(f"Fold {fold}: F1 = {f1:.4f}")

print(f"\nMean CV F1: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")
print(f"Test F1: 0.7095")
print("\nGood! CV and test scores are close - no major overfitting.")

Running 5-Fold Cross-Validation...
Fold 1: F1 = 0.6940
Fold 2: F1 = 0.6915
Fold 3: F1 = 0.6944
Fold 4: F1 = 0.6820
Fold 5: F1 = 0.6912

Mean CV F1: 0.6906 (+/- 0.0045)
Test F1: 0.7095

Good! CV and test scores are close - no major overfitting.


In [35]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 300, 700),
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.2),
        'depth': trial.suggest_int('depth', 4, 8),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 100),
        'class_weights': class_weight_dict,
        'cat_features': cat_feature_indices,
        'random_seed': 42,
        'verbose': False
    }
    
    model = CatBoostClassifier(**params)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='f1_macro', n_jobs=-1)
    return score.mean()

print("Starting hyperparameter optimization (30 trials)...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("\n" + "="*50)
print("BEST PARAMETERS:")
print(study.best_params)
print(f"Best CV F1: {study.best_value:.4f}")

# Train final model with best params
best_params = study.best_params
best_params.update({
    'class_weights': class_weight_dict,
    'cat_features': cat_feature_indices,
    'random_seed': 42,
    'verbose': 100
})

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train, y_train)

y_pred_final = final_model.predict(X_test)
print("\n" + "="*50)
print("FINAL TUNED MODEL:")
print("="*50)
print(f"Accuracy: {accuracy_score(y_test, y_pred_final):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_final, average='macro'):.4f}")
print("\n", classification_report(y_test, y_pred_final))

[I 2025-12-27 21:02:06,311] A new study created in memory with name: no-name-d11bb64e-a5db-481d-86ac-aa947cd26af3


Starting hyperparameter optimization (30 trials)...


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-12-27 21:03:14,391] Trial 0 finished with value: 0.6907800660368619 and parameters: {'iterations': 626, 'learning_rate': 0.09453529012808881, 'depth': 5, 'l2_leaf_reg': 6.4194186812966665, 'min_data_in_leaf': 77}. Best is trial 0 with value: 0.6907800660368619.
[I 2025-12-27 21:04:51,673] Trial 1 finished with value: 0.6944373150428857 and parameters: {'iterations': 576, 'learning_rate': 0.07984126706847643, 'depth': 8, 'l2_leaf_reg': 1.6600071994492231, 'min_data_in_leaf': 87}. Best is trial 1 with value: 0.6944373150428857.
[I 2025-12-27 21:05:42,780] Trial 2 finished with value: 0.6899367389950243 and parameters: {'iterations': 614, 'learning_rate': 0.11162756608980867, 'depth': 5, 'l2_leaf_reg': 4.373883459158026, 'min_data_in_leaf': 63}. Best is trial 1 with value: 0.6944373150428857.
[I 2025-12-27 21:06:14,306] Trial 3 finished with value: 0.6903099994332732 and parameters: {'iterations': 503, 'learning_rate': 0.08780675239598174, 'depth': 4, 'l2_leaf_reg': 7.009437260496

In [21]:
import mlflow
import mlflow.catboost
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import json
import matplotlib.pyplot as plt
import seaborn as sns


# Set experiment name
mlflow.set_experiment("Profit_Prediction_Experiment")

# Start MLflow run
with mlflow.start_run(run_name="catboost_3class_final"):
    
    print("="*60)
    print("STEP 1: DATA PREPARATION & LOGGING")
    print("="*60)
    
    # Create 3 classes
    def profit_bucket_3class(x):
        if x < 0:
            return "loss"
        elif x < 0.25:
            return "profit_low"
        else:
            return "profit_high"
    
    df['profit_class_3'] = df['profit_margin'].apply(profit_bucket_3class)
    
    # Log raw dataset info
    mlflow.log_param("total_samples", len(df))
    mlflow.log_param("n_features_raw", df.shape[1])
    mlflow.log_param("class_distribution", df['profit_class_3'].value_counts().to_dict())
    
    # Log class boundaries
    mlflow.log_param("loss_boundary", "< 0")
    mlflow.log_param("profit_low_boundary", "0 to 0.25")
    mlflow.log_param("profit_high_boundary", "> 0.25")
    
    # Save raw dataset sample
    df.head(100).to_csv("data_sample.csv", index=False)
    mlflow.log_artifact("data_sample.csv", "data")
    
    # Prepare features
    X = df.drop(columns=["profit_margin", "profit_class", "profit_class_quantile", 
                         "profit_class_correct", "profit_class_3", "product_name", "store_name"])
    y = df["profit_class_3"]
    
    # Log feature names
    feature_names = X.columns.tolist()
    mlflow.log_param("n_features_final", len(feature_names))
    with open("feature_names.json", "w") as f:
        json.dump(feature_names, f, indent=2)
    mlflow.log_artifact("feature_names.json", "features")
    
    # Log categorical features
    cat_features = ['product_id', 'category', 'brand', 'store_id', 'store_location',
                    'promotion_type', 'day_of_week', 'season']
    with open("categorical_features.json", "w") as f:
        json.dump(cat_features, f, indent=2)
    mlflow.log_artifact("categorical_features.json", "features")
    
    print("✓ Dataset info logged")
    
    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )
    
    # Log split info
    mlflow.log_param("train_size", len(X_train))
    mlflow.log_param("val_size", len(X_val))
    mlflow.log_param("test_size", len(X_test))
    mlflow.log_param("test_split_ratio", 0.15)
    mlflow.log_param("random_state", 42)
    mlflow.log_param("stratify", True)
    
    print("="*60)
    print("STEP 2: CLASS WEIGHTS & PREPROCESSING")
    print("="*60)
    
    # Compute class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weight_dict = dict(zip(np.unique(y_train), class_weights))
    
    # Log class weights
    for cls, weight in class_weight_dict.items():
        mlflow.log_param(f"class_weight_{cls}", float(weight))
    
    # Get categorical indices
    cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_features]
    mlflow.log_param("categorical_indices", cat_feature_indices)
    
    print("✓ Class weights computed and logged")
    
    print("="*60)
    print("STEP 3: MODEL TRAINING")
    print("="*60)
    
    # Model hyperparameters (use your best params from Optuna)
    params = {
        'iterations': 576,
        'learning_rate': 0.07984126706847643,
        'depth': 8,
        'l2_leaf_reg': 1.6600071994492231,
        'min_data_in_leaf': 87,
        'class_weights': class_weight_dict,
        'cat_features': cat_feature_indices,
        'random_seed': 42,
        'verbose': 100
    }
    
    # Log all hyperparameters
    for key, value in params.items():
        if key not in ['class_weights', 'cat_features']:
            mlflow.log_param(key, value)
    
    # Train model
    model = CatBoostClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=100
    )
    
    print("✓ Model trained")
    
    print("="*60)
    print("STEP 4: MODEL EVALUATION & METRICS")
    print("="*60)
    
    # Predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    # Calculate metrics for all sets
    train_acc = accuracy_score(y_train, y_train_pred)
    train_f1 = f1_score(y_train, y_train_pred, average='macro')
    
    val_acc = accuracy_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred, average='macro')
    
    test_acc = accuracy_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred, average='macro')
    
    # Log metrics
    mlflow.log_metric("train_accuracy", train_acc)
    mlflow.log_metric("train_f1_macro", train_f1)
    mlflow.log_metric("val_accuracy", val_acc)
    mlflow.log_metric("val_f1_macro", val_f1)
    mlflow.log_metric("test_accuracy", test_acc)
    mlflow.log_metric("test_f1_macro", test_f1)
    
    # Per-class metrics for test set
    report = classification_report(y_test, y_test_pred, output_dict=True)
    for class_name in ['loss', 'profit_low', 'profit_high']:
        if class_name in report:
            mlflow.log_metric(f"test_{class_name}_precision", report[class_name]['precision'])
            mlflow.log_metric(f"test_{class_name}_recall", report[class_name]['recall'])
            mlflow.log_metric(f"test_{class_name}_f1", report[class_name]['f1-score'])
    
    print(f"Train Accuracy: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"Val Accuracy: {val_acc:.4f}, F1: {val_f1:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}, F1: {test_f1:.4f}")
    
    # Save classification report
    report_text = classification_report(y_test, y_test_pred)
    with open("classification_report.txt", "w") as f:
        f.write(report_text)
    mlflow.log_artifact("classification_report.txt", "evaluation")
    
    print("="*60)
    print("STEP 5: CONFUSION MATRIX & VISUALIZATIONS")
    print("="*60)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_test_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['loss', 'profit_high', 'profit_low'],
                yticklabels=['loss', 'profit_high', 'profit_low'])
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig("confusion_matrix.png", dpi=150)
    mlflow.log_artifact("confusion_matrix.png", "visualizations")
    plt.close()
    
    # Feature importance
    feature_importance = model.get_feature_importance()
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    # Save feature importance
    importance_df.to_csv("feature_importance.csv", index=False)
    mlflow.log_artifact("feature_importance.csv", "features")
    
    # Plot top 15 features
    plt.figure(figsize=(10, 8))
    top_features = importance_df.head(15)
    plt.barh(range(len(top_features)), top_features['importance'])
    plt.yticks(range(len(top_features)), top_features['feature'])
    plt.xlabel('Importance')
    plt.title('Top 15 Feature Importances')
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig("feature_importance.png", dpi=150)
    mlflow.log_artifact("feature_importance.png", "visualizations")
    plt.close()
    
    print("✓ Visualizations created and logged")
    
    print("="*60)
    print("STEP 6: MODEL SAVING")
    print("="*60)
    
    # Log model with MLflow
    mlflow.catboost.log_model(
        model,
        "model",
        registered_model_name="profit_classifier_catboost"
    )
    
    # Also save as native CatBoost format
    model.save_model("catboost_model.cbm")
    mlflow.log_artifact("catboost_model.cbm", "model_native")
    
    print("✓ Model saved")
    
    print("="*60)
    print("STEP 7: ADDITIONAL ARTIFACTS FOR DOCKER")
    print("="*60)
    
    # Save preprocessing info
    preprocessing_info = {
        "feature_names": feature_names,
        "categorical_features": cat_features,
        "categorical_indices": cat_feature_indices,
        "class_names": ['loss', 'profit_high', 'profit_low'],
        "class_weights": {k: float(v) for k, v in class_weight_dict.items()},
        "profit_boundaries": {
            "loss": "< 0",
            "profit_low": "0 to 0.25",
            "profit_high": "> 0.25"
        }
    }
    
    with open("preprocessing_config.json", "w") as f:
        json.dump(preprocessing_info, f, indent=2)
    mlflow.log_artifact("preprocessing_config.json", "config")
    
    # Save model signature for inference
    model_signature = {
        "input_features": feature_names,
        "output_classes": ['loss', 'profit_high', 'profit_low'],
        "model_type": "CatBoostClassifier",
        "model_version": "1.0"
    }
    
    with open("model_signature.json", "w") as f:
        json.dump(model_signature, f, indent=2)
    mlflow.log_artifact("model_signature.json", "config")
    
    # Save requirements.txt for Docker
    requirements = """catboost==1.2
pandas==2.0.0
numpy==1.24.0
scikit-learn==1.3.0
mlflow==2.9.0
"""
    with open("requirements.txt", "w") as f:
        f.write(requirements)
    mlflow.log_artifact("requirements.txt", "docker")
    
    print("✓ Docker artifacts saved")
    
    # Get run ID for reference
    run_id = mlflow.active_run().info.run_id
    
    print("="*60)
    print("MLflow LOGGING COMPLETE!")
    print("="*60)
    print(f"Run ID: {run_id}")
    print(f"Experiment: profit_classification_experiment")
    print(f"Model registered as: profit_classifier_catboost")
    print("\nTo view results:")
    print("  mlflow ui")
    print("  Then open: http://localhost:5000")
    print("="*60)

print("\n✅ ALL DONE! Everything logged to MLflow.")
print("\nNext steps:")
print("1. Run 'mlflow ui' to view results")
print("2. Use the run_id to load the model later")
print("3. Check mlruns/ folder for all artifacts")

STEP 1: DATA PREPARATION & LOGGING
✓ Dataset info logged
STEP 2: CLASS WEIGHTS & PREPROCESSING
✓ Class weights computed and logged
STEP 3: MODEL TRAINING
0:	learn: 1.0361173	test: 1.0356783	best: 1.0356783 (0)	total: 70.6ms	remaining: 40.6s
100:	learn: 0.5467334	test: 0.5699004	best: 0.5699004 (100)	total: 7.56s	remaining: 35.5s
200:	learn: 0.4995275	test: 0.5669162	best: 0.5667703 (190)	total: 15.7s	remaining: 29.2s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5667702643
bestIteration = 190

Shrink model to first 191 iterations.
✓ Model trained
STEP 4: MODEL EVALUATION & METRICS
Train Accuracy: 0.7109, F1: 0.7068
Val Accuracy: 0.7076, F1: 0.7043
Test Accuracy: 0.7109, F1: 0.7047
STEP 5: CONFUSION MATRIX & VISUALIZATIONS


2025/12/31 20:58:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✓ Visualizations created and logged
STEP 6: MODEL SAVING


2025/12/31 20:58:24 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/31 20:58:24 INFO mlflow.store.db.utils: Updating database tables
2025/12/31 20:58:24 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2025/12/31 20:58:24 INFO alembic.runtime.migration: Will assume non-transactional DDL.
Successfully registered model 'profit_classifier_catboost'.
Created version '1' of model 'profit_classifier_catboost'.


✓ Model saved
STEP 7: ADDITIONAL ARTIFACTS FOR DOCKER
✓ Docker artifacts saved
MLflow LOGGING COMPLETE!
Run ID: 85fc8c6fe9254731924a87b76c100212
Experiment: profit_classification_experiment
Model registered as: profit_classifier_catboost

To view results:
  mlflow ui
  Then open: http://localhost:5000

✅ ALL DONE! Everything logged to MLflow.

Next steps:
1. Run 'mlflow ui' to view results
2. Use the run_id to load the model later
3. Check mlruns/ folder for all artifacts


In [28]:
import mlflow
import mlflow.pyfunc
import mlflow.catboost
import pandas as pd
import numpy as np
import json

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from sklearn.base import BaseEstimator, TransformerMixin


# ===============================
# 1️⃣ PROFIT BUCKET FUNCTION
# ===============================
def profit_bucket_3class(x):
    if x < 0:
        return "loss"
    elif x < 0.25:
        return "profit_low"
    else:
        return "profit_high"


# ===============================
# 2️⃣ FEATURE SELECTOR TRANSFORMER
# ===============================
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.feature_names]


# ===============================
# 3️⃣ MLflow PIPELINE MODEL
# ===============================
class ProfitPipelineModel(mlflow.pyfunc.PythonModel):

    def __init__(self, model, feature_names):
        self.model = model
        self.feature_names = feature_names

    def predict(self, context, model_input):
        X = model_input[self.feature_names]
        return self.model.predict(X)


# ===============================
# 4️⃣ START EXPERIMENT
# ===============================
mlflow.set_experiment("Profit_margin_Prediction_Experiment")

with mlflow.start_run(run_name="catboost_pipeline_final"):

    # ===============================
    # DATA PREPARATION
    # ===============================
    df["profit_class_3"] = df["profit_margin"].apply(profit_bucket_3class)

    X = df.drop(columns=[
        "profit_margin",
        "profit_class",
        "profit_class_quantile",
        "profit_class_3",
        "profit_class_correct",  
        "product_name",
        "store_name"
    ])

    y = df["profit_class_3"]

    feature_names = X.columns.tolist()

    # ===============================
    # LOG DATASET METADATA
    # ===============================
    mlflow.log_param("total_samples", len(df))
    mlflow.log_param("n_features", len(feature_names))
    mlflow.log_param("class_distribution", y.value_counts().to_dict())

    # ===============================
    # CATEGORICAL FEATURES
    # ===============================
    cat_features = [
        "product_id",
        "category",
        "brand",
        "store_id",
        "store_location",
        "promotion_type",
        "day_of_week",
        "season"
    ]

    cat_feature_indices = [feature_names.index(col) for col in cat_features]

    # ===============================
    # TRAIN / VAL / TEST SPLIT
    # ===============================
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=42
    )

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    # ===============================
    # CLASS WEIGHTS
    # ===============================
    class_weights = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = dict(zip(np.unique(y_train), class_weights))

    # ===============================
    # MODEL PARAMETERS (UNCHANGED)
    # ===============================
    params = {
        "iterations": 576,
        "learning_rate": 0.07984126706847643,
        "depth": 8,
        "l2_leaf_reg": 1.66,
        "min_data_in_leaf": 87,
        "class_weights": class_weight_dict,
        "cat_features": cat_feature_indices,
        "random_seed": 42,
        "verbose": 100
    }

    # ===============================
    # TRAIN MODEL
    # ===============================
    model = CatBoostClassifier(**params)

    model.fit(
        X_train,
        y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=100
    )

    # ===============================
    # EVALUATION
    # ===============================
    y_test_pred = model.predict(X_test)

    mlflow.log_metric("test_accuracy", accuracy_score(y_test, y_test_pred))
    mlflow.log_metric("test_f1_macro", f1_score(y_test, y_test_pred, average="macro"))

    report = classification_report(y_test, y_test_pred)
    with open("classification_report.txt", "w") as f:
        f.write(report)
    mlflow.log_artifact("classification_report.txt", "evaluation")

    # ===============================
    # SAVE PIPELINE (CRITICAL PART)
    # ===============================
    pipeline_model = ProfitPipelineModel(
        model=model,
        feature_names=feature_names
    )

    mlflow.pyfunc.log_model(
        artifact_path="profit_classifier_pipeline",
        python_model=pipeline_model,
        registered_model_name="profit_classifier_pipeline"
    )

    # ===============================
    # SAVE PREPROCESSING CONFIG
    # ===============================
    preprocessing_config = {
        "feature_names": feature_names,
        "categorical_features": cat_features,
        "categorical_indices": cat_feature_indices,
        "class_labels": ["loss", "profit_low", "profit_high"]
    }

    with open("preprocessing_config.json", "w") as f:
        json.dump(preprocessing_config, f, indent=2)

    mlflow.log_artifact("preprocessing_config.json", "config")

    # ===============================
    # REQUIREMENTS FOR DOCKER
    # ===============================
    requirements = """mlflow==2.9.0
catboost==1.2
pandas==2.0.0
numpy==1.24.0
scikit-learn==1.3.0
"""

    with open("requirements.txt", "w") as f:
        f.write(requirements)

    mlflow.log_artifact("requirements.txt", "docker")

    print("✅ PIPELINE MODEL SAVED & REGISTERED")


c:\Users\fatim\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


0:	learn: 1.0361173	test: 1.0356783	best: 1.0356783 (0)	total: 175ms	remaining: 1m 40s
100:	learn: 0.5467334	test: 0.5699004	best: 0.5699004 (100)	total: 11.6s	remaining: 54.4s
200:	learn: 0.4995275	test: 0.5669162	best: 0.5667703 (190)	total: 21.6s	remaining: 40.4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.5667702637
bestIteration = 190

Shrink model to first 191 iterations.


2026/01/01 01:39:34 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ PIPELINE MODEL SAVED & REGISTERED


Registered model 'profit_classifier_pipeline' already exists. Creating a new version of this model...
Created version '3' of model 'profit_classifier_pipeline'.


In [26]:
# Before creating X, check for any profit-related columns
print("All columns:")
print(df.columns.tolist())

print("\nColumns with 'profit' in name:")
profit_cols = [col for col in df.columns if 'profit' in col.lower()]
print(profit_cols)

# Check what's in X after dropping
print("\nColumns in X:")
print(X.columns.tolist())

# Check for any object/string columns that shouldn't be there
print("\nObject columns in X:")
print(X.select_dtypes(include='object').columns.tolist())

All columns:
['product_id', 'product_name', 'category', 'brand', 'store_id', 'store_name', 'store_location', 'base_price', 'discount_rate', 'promotion_type', 'day_of_year', 'month', 'day_of_week', 'season', 'is_holiday', 'avg_units_sold_30d', 'avg_customers_30d', 'profit_margin', 'profit_class', 'profit_class_quantile', 'profit_class_3', 'profit_class_correct']

Columns with 'profit' in name:
['profit_margin', 'profit_class', 'profit_class_quantile', 'profit_class_3', 'profit_class_correct']

Columns in X:
['product_id', 'category', 'brand', 'store_id', 'store_location', 'base_price', 'discount_rate', 'promotion_type', 'day_of_year', 'month', 'day_of_week', 'season', 'is_holiday', 'avg_units_sold_30d', 'avg_customers_30d', 'profit_class_correct']

Object columns in X:
['product_id', 'category', 'brand', 'store_id', 'store_location', 'promotion_type', 'day_of_week', 'season', 'profit_class_correct']
